In [1]:
!pip install pandas numpy matplotlib statsmodels pandas_datareader datetime yfinance sklearn PyPortfolioOpt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 59.8 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 6.3 MB/s eta 0:00:00
  Obtaining dependency information for datetime from https://files.pythonhosted.org/packages/95/88/3b9d4042b396221a132180b392ab2a174031a6fb579f7927f3909fc183a7/DateTime-5.2-py3-none-any.whl.metadata
  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/e6/b3/388ab967a387cc92926f70e97688dd9a7189b29a0773db815ffc5289e2b5/yfinance-0.2.31-py2.py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 32.7 MB/s eta 0:00:00
  Obtaining dependency information for lxml from https://files.pythonhosted.org/packages/44/1b/0771c38e65ad23e25368b5e07c920054774b8d12477a4fad116bf500de73/lxml-4.9.3-cp38-cp38-manylinux_2_28_x86_64.whl.metadata
  Obtaini

In [6]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=fae2da470828418eea659ba6fc17bf4cdfad7251655a09bc8c18cec95a33d778
  Stored in directory: /home/studio-lab-user/.cache/pip/wheels/54/4a/75/06b8e63fce6f6d2c1baae5c208edb18eca128407b0c96e1153
Successfully built pandas_ta


In [7]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings

In [8]:
warnings.filterwarnings('ignore')

In [15]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

In [16]:
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

In [17]:
symbol_list = sp500['Symbol'].unique().tolist()

In [18]:
end_date = '2023-10-30'
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

In [19]:
df = yf.download(tickers=symbol_list,
                 start=start_date,
                 end=end_date)

[*********************100%%**********************]  503 of 503 completed


In [27]:
sdf = df.stack()

In [28]:
sdf.index.names = ['date', 'ticker']

In [31]:
sdf.columns = sdf.columns.str.lower()

In [35]:
sdf['garman_klass_vol'] = ((np.log(sdf['high']) - np.log(sdf['low']))**2)/2-(2 * np.log(2)-1) * (np.log(sdf['adj close']) - np.log(sdf['open']))**2

In [43]:
sdf['rsi'] = sdf.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

In [57]:
sdf['bb_low'] = sdf.groupby(level=1)['adj close'].transform(lambda x : pandas_ta.bbands(close=np.log1p(x), lenght=20).iloc[:,0])
sdf['bb_mid'] = sdf.groupby(level=1)['adj close'].transform(lambda x : pandas_ta.bbands(close=np.log1p(x), lenght=20).iloc[:,1])
sdf['bb_high'] = sdf.groupby(level=1)['adj close'].transform(lambda x : pandas_ta.bbands(close=np.log1p(x), lenght=20).iloc[:,2])

In [58]:
sdf

adj close       close  ...    bb_mid   bb_high
date       ticker                          ...                    
2015-11-02 A        36.128700   38.590000  ...       NaN       NaN
           AAL      44.313286   46.470001  ...       NaN       NaN
           AAPL     27.498983   30.295000  ...       NaN       NaN
           ABBV     44.862263   63.380001  ...       NaN       NaN
           ABT      39.108418   45.430000  ...       NaN       NaN
...                       ...         ...  ...       ...       ...
2023-10-27 YUM     119.440002  119.440002  ...  4.791089  4.801366
           ZBH     103.190002  103.190002  ...  4.653832  4.670000
           ZBRA    207.179993  207.179993  ...  5.324978  5.353140
           ZION     29.450001   29.450001  ...  3.430305  3.452712
           ZTS     156.029999  156.029999  ...  5.095077  5.150077

[994050 rows x 11 columns]